***CATEGORY THEORY WITH JULIA***
-----

# Enforcing Types with Morphisms

This is the "applied" part of Category Theory, focusing on programming.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAP/PhDThesis/notes/FunctionalProgrammingCategoryTheory/julia`


## 1. Creating Morphisms in Julia

We start be defining the composition operator. This is actually already present in Julia via `\circ`. But here we'll implement it
again using the code from Julia's Base.

The implementation relys on function-like objects that are structs that can be called, similar to a function.

In [2]:
struct Comp{O,I} <: Function
    outer::O
    inner::I
    Comp{O, I}(outer, inner) where {O, I} = new{O, I}(outer, inner)
    Comp(outer, inner) = new{Core.Typeof(outer),Core.Typeof(inner)}(outer, inner)
end

function (c::Comp)(x...; kw...)
    return c.outer(c.inner(x...; kw...))
end

In the code above, the last two lines in the `struct`are constructors, where the first one allow us to do
`Comp{Function, Function}(g, f)` and for the second we do `Comp(g,f)`. In the last line we are defining a function
to be dispatched on `Comp`, but this function is nameless.

With this, our composition is now working.

In [3]:
Comp{Function, Function}(x->x, y->y^2)
Comp(x->x, y->y^2)

(::Comp{var"#6#8", var"#7#9"}) (generic function with 1 method)

In [4]:
id = x -> x
sq = x -> x^2
Comp(id,sq)(2)

4

The current implementation does not enforce anything related to domain and codomain.
Let's create a struct for that, called morphism.

In [5]:
struct Morphism1
    f::Function
    dom::Type
    codom::Type
end

In [6]:
m = Morphism1(x->x^2, Real, Real)
m.f(2)

4

This works. But we cannot dispatch our morphisms based on their domain and codomain.
Perhaps it's better to use parametric types.

Thus, let's reformulate our struct and make it similar to `Comp`.
Let's define a function such
that our struct would work like a function without having to use the `.f`.
 

In [7]:
function getfunctiontype(f, methodindex=1)
    m = collect(methods(f))[methodindex]
    dom = Tuple([m.sig.types[2:end]...])
    codom = Base.code_typed(f, dom)[1].second
    
    if length(dom) == 1
        dom = dom[1]
    end
    return dom,codom
end

function F(x,y::Real)::Int
    return x
end


getfunctiontype(F)

((Any, Real), Int64)

In [8]:
struct Morphism{D,C}
    f::Function
end

function Morphism(f::Function)
    D,C = getfunctiontype(f)
    if typeof(D) <: Tuple
        D = Tuple{D...}
    end
    Morphism{D,C}(f)
end

function (m::Morphism)(x)
    m.f(x)
end

function ff(x)
    x^2
end
m1 = Morphism(F)
m2 = Morphism(x->x^2)

Morphism{Any, Any}(var"#16#17"())

In [9]:
m2(2)

4

Let's create a macro to ease the way we define morphisms. 

In [10]:
using MacroTools

function extractypes(dex)
    t = map(dex[:args]) do a
        if a isa Symbol
            return Any
        else
            return eval(a.args[2])
        end
    end
    length(t) == 1 ? t[1] : Tuple{t...}
end

macro morphism(ex)
    dex = MacroTools.splitdef(ex)
    dom = extractypes(dex)
    codom = Any
    if :rtype in keys(dex)
        codom = eval(dex[:rtype])
    end
    
    name = esc(dex[:name])
    
    return quote
        $name = Morphism{$dom, $codom}($ex)
    end
end

@morphism (macro with 1 method)

In [11]:
@morphism function m(x::Int, y::Real)::Int
    return x^2
end

Morphism{Tuple{Int64, Real}, Int64}(var"#32#m")

Nice! Let's write two helper functions to get the domain and codomain. 

In [12]:
dom(m::Morphism) = typeof(m).parameters[1]
codom(m::Morphism) = typeof(m).parameters[2];

Now that we can construct our morphisms, let's implement the composition
of such morphisms. In our case, the composition must restrict to matching types.

In [13]:
function Base.:∘(f::Morphism, g::Morphism)
    if codom(g) === dom(f)
        return Morphism{dom(g), codom(f)}(f.f ∘ g.f)
    end
    error("Codomain of $f does not match domain of $g")
end

In [14]:
@morphism function g(x::Int)::Int
   x 
end
@morphism function f(x::Int)::Real
   √x 
end

Morphism{Int64, Real}(var"#37#f")

In [15]:
(f∘g)(4)

2.0

## 2 Type enforcing with FunctionWrappers 

We've implemented our Morphism in order to enforce types in functions.
Our implementation was more for learning, and not focused on optimization.

A "pro" implementation of this is done in the FunctionWrappers.jl package.
Yet, the goal of function wrappers is to do composition of functions an so on. Hence,
we proceed with our morphism implementaion.

Let's just finish this by showing an example of usage of FunctionWrappers.jl.

In [28]:
import FunctionWrappers: FunctionWrapper

In [32]:
f = FunctionWrapper{Float64,Tuple{Int}}(x->x^0.5)
typeof(f)

FunctionWrapper{Float64, Tuple{Int64}}

## 3. Functions as Exponential Objects

We've seen in section 1 on Types (the other notebook),
that we can do algebra on types. For example, we can definte `Int × Int` which
is equivalent to `Tuple{Int,Int}`, and we can do `String + Int` which is equivalent
to `Union{String,Int}`. What about exponentiation?

First, remember that not every category has products, meaning, it's not always
true that for two objects $A$ and $B$ in $\mathcal C$, that $A \times B$ is also
an object in $\mathcal C$. The same is true for exponential objects.

Moreover, whenever a category has all finite products and exponential objects,
this category is said to be a **Cartesian Closed Category**.

In set theory, the set $Z^Y$ is the set of all functions from $Y$ to $Z$. Hence,
analogously, for a small category $\mathcal C$, an object $Z^Y$ would be the
set of morphisms from $Y$ to $Z$, i.e. $\text{Hom}(Y,Z)$.

Hence, in programming, we can think of `Int^String` as the type consisting of
functions that take strings as arguments and return integers.

Yet, as we've said, in Julia we do not have these types, meaning,
Julia does not enforce that our function always returns a string.

Of course, there are ways around it, as we've shown we our implementation of morphisms.

## 4. Currying

In [101]:
f(x,y) = x + y
curry(f::Function,y) = x->f(x,y)

curry (generic function with 2 methods)

In [105]:
f(1,2)
curry(f,1)(2)

3